In [1]:
%pylab inline
import scipy
import itertools

from pyiga import bspline, assemble, vform, geometry, vis, solvers

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

# assemble matrix
from scipy.sparse import coo_matrix, block_diag, bmat
from scipy.sparse import bsr_matrix

from geo_annulus import *
#from multipatch_block_handler import *
from plots import *
from ass_nonlin_el import *
from solver import *
from line_search import *


Populating the interactive namespace from numpy and matplotlib
OK


In [2]:
p = 3  # spline degree 
x_el= 24
y_el= 4
n_el = (x_el, y_el) #  isotropic material
# displacement space: degree p,   continuity p-1
multi= 1
kvs_u = tuple(bspline.make_knots(p, 0.0, 1.0, n, mult=multi) for n in n_el) # or : mult=2
#m_u = tuple(kv.numdofs for kv in kvs_u)

In [37]:
r_in = 1.93 #mm
r_out = 2.25 #mm
geos = [

        geometry.quarter_annulus(r1=r_in, r2=r_out),
        geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(3*np.pi/2),
        geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(np.pi),
        geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(np.pi/2)
    ]


In [38]:
fig, ax = plt.subplots(figsize=(8,6))
for g in geos:
    vis.plot_geo(g, gridx=np.linspace(0, 1, x_el), gridy=np.linspace(0, 1, y_el))
ax.axis('scaled');


In [39]:

def quarter_annulus(r1=1.0, r2=2.0):
    """A NURBS representation of a quarter annulus in the first quadrant.
    The 'bottom' and 'top' boundaries (with respect to the reference domain)
    lie on the x and y axis, respectively.

    Args:
        r1 (float): inner radius
        r2 (float): outer radius

    Returns:
        :class:`NurbsFunc` 2D geometry
    """
    kvx = bspline.make_knots(1, 0.0, 1.0, 1)
    kvy = bspline.make_knots(2, 0.0, 1.0, 1)

    coeffs = np.array([
            [[ r1, 0.0, 1.0],
             [ r2, 0.0, 1.0]],
            [[ r1+0.3,  r1+0.3, 1.0 ],
             [ r2,  r2, 1.0 ]],
            [[0.0,  r1, 1.0],
             [0.0,  r2, 1.0]],
    ])
    return geometry.NurbsFunc((kvy,kvx), coeffs, weights=None)

In [40]:
g= quarter_annulus(r1=r_in, r2=r_out)
fig, ax = plt.subplots(figsize=(8,6))
vis.plot_geo(g, gridx=np.linspace(0, 1, x_el), gridy=np.linspace(0, 1, y_el))
ax.axis('scaled');

In [41]:
geos = [

        quarter_annulus(r1=r_in, r2=r_out),
        geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(3*np.pi/2),
        geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(np.pi),
        geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(np.pi/2)
    ]

In [42]:
fig, ax = plt.subplots(figsize=(8,6))
for g in geos:
    vis.plot_geo(g, gridx=np.linspace(0, 1, x_el), gridy=np.linspace(0, 1, y_el))
ax.axis('scaled');


In [43]:
patches_u = [(kvs_u, g) for g in geos]

# Here we auto-detect the interfaces between the patches.
MP_u = assemble.Multipatch(patches_u, automatch=True)

In [44]:
## set blocks for x and y component

# Multipatch objects for all variables
MP_block = multipatch_block_handler( [MP_u, MP_u] )


In [45]:
# source term f 
def f(x, y): return (0.0, 0.0)

#Neumann BC
#def gN(x, y): return (x, -y)  #outer pressure
def gN(x, y): 
    return (x/r_in*l_val, y/r_in*l_val)  #inner  pressure, normalized vector

# define Dirichlet boundary function 
def g_zero(x, y): return (0.0, 0.0)


# Robin BC
def g_robin(x, y): 
    return (0)
    #return ( 1-(x/r_out+ y/r_out)**2) 
    #return ((x/r_out*p_out)**2 +(y/r_out*p_out)**2) # scalar value
    

In [46]:

# set up Dirichlet boundary conditions
bc = MP_block.compute_dirichlet_bcs([
    (1, 'right', g_zero ) 
])


In [47]:
# define constant spline functions for integration
kvs_j = tuple(bspline.make_knots(0, 0.0, 1.0, n, mult=multi) for n in n_el) # constant basis vector for integration

In [48]:
#incremental loading
# pressure: 13.33 kPa --> 100mmHg
#           10.664 kPa --> 80mmHg
# p_int:    2.666 kPa ---> 20mmHg
maxload = 2.666e-3 #1.5e-3 #13.33*1e-3 #1.5e-2 #2e-3
nsteps= 3
loading= np.linspace(maxload/nsteps, maxload, nsteps)

#p_out = 10.664*1e-3
#p_inner = 13.33*1e-3
#loading_out= np.linspace(p_out/nsteps, p_out, nsteps)
#loading_in= np.linspace(p_inner/nsteps, p_inner, nsteps)
p_out= 2.666e-3

In [49]:
alpha =2.5e-2# 1.5e3# 1e0 # 1e4 max penalization!
robin_data = [
            #(0, 'right', g_robin, alpha),
             #(1, 'right', g_robin, alpha),
             (2, 'right', g_robin, alpha),
             (3, 'right', g_robin, alpha)]# instead of Dirichlet-bdc, outer bd fixed

#robin_data = [(1, 'right', g_zero, 1)] # instead of Dirichlet-bdc, outer bd fixed

In [50]:
# assemble Robin-matrix
AR = ass_Robin(MP_block, robin_data)

In [51]:
def J(u):
    return ass_energy(u, MP_block, kvs_j, neu_data, robin_data)


def grad_J(u):
    return -ass_rhs_RN(u, MP_block, neu_data, AR)


In [52]:
# solve linearized system #Robin instead of Dirichlet, + Neumann force (inner pressure)
### first iteration ###
XP = MP_block.patch_to_global(0) #p=0
dd = shape(XP)[0]

#initial value
u= np.zeros(dd)
solution= []

# set Neumann boundary force (via incremental loading) 

### Linear elasticity for largest loading
l_val = loading[-1] # take last loading value
neu_data = [(0,'left',gN, l_val), (1,'left',gN, l_val), (2,'left',gN, l_val), (3,'left',gN, l_val)] # set neumann

### first iteration ###
A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR) # with Robin-matrix
#S,sig= ass_cauchystress(u, MP_block) 

M = ass_mass(MP_block)
Minv = make_solver(M)

r0 = np.transpose(b).dot(Minv.dot(b)) #L2-norm
print('Residual0 =', r0)
print('Energy0   =', J(u))

#u_d = make_solver_orig(A, symmetric=True, spd=True).dot(b) 
u_d = make_solver(A).dot(b) 
u += u_d

#cauchystress= make_solver(sig).dot(S) 
#A, b = ass_nonlinsystem(u, MP_block, neu_data, AR) 
#print('rhs:', b)
b= ass_rhs_RN( u, MP_block, neu_data, AR)
#print('rhs:', b)

r = np.transpose(b).dot(Minv.dot(b)) #L2-norm
print('Residual  =', r)
print('Energy    =', J(u))

w= np.inner(np.transpose(u_d),b)
err = np.sqrt(np.abs(w))
print('Error=', err)

###-----------------###--------------------###
solution = [u]
# norm of delta u in first step
normed_du0= np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
###----------------###---------------------###

# print deformation plot (after first iteration)
get_defplot(u, patches_u, kvs_u, MP_u, n_el)

Residual0 = 0.013654805074382386
Energy0   = 0.0
Residual  = 129099.22729153631
Energy    = 11.281844647840515
Error= 6.707613398543985


In [53]:
get_defplot_scalar(local_vol(u, MP_block), patches_u, kvs_u, MP_u, n_el, geos)

In [54]:
# solve linearized variational problem - iterative, without line-search

#initial value
u= np.zeros(dd)

max_err=1e-11

solutions = []
stepsizes = [] 
ud_array = []
iter_counts = []
# incremental loading # ----------------------------------
for t in range(len(loading)): # time steps
    print(" \n \n {}. loading: {} \n".format(t+1,loading[t]))
    
    # set Neumann data via incremental loading
    l_val = loading[t]
    neu_data = [(0,'left',gN, l_val), (1,'left',gN, l_val), (2,'left',gN, l_val), (3,'left',gN, l_val)] # set neumann

    count = 0
    while count <len(loading)-2:
        count+=1
        print(count)
        A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
        r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
        print('Residual =',r)
        print('Energy   =',J(u))
       
        # # solve system # #
        #u_d = make_solver(A).dot(b) 
        u_d = make_solver(A).dot(b)
        u += u_d  
        
        w= np.inner(np.transpose(u_d),b)
        err = np.sqrt(np.abs(w))
        print('Error=', err)
        
        ud_array.append(u_d)
        normed_du = np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
        #normed_du = np.linalg.norm(u_d)/np.sqrt(len(u_d))
        stepsize_du = normed_du # times alpha
        stepsizes.append(stepsize_du)
        

    solutions.append(np.array(u))
    iter_counts.append(count)
    count=0

# solve solution exact on max. loading 
while True:
    A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
    r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm

    print('residual =', r)
    print('energy   =', J(u))
    
    # # solve system # #
    u_d = make_solver(A).dot(b)
    
    w= np.inner(np.transpose(u_d),b)
    err = np.sqrt(np.abs(w))
    print('Error=', err)
    
    if abs(err) < max_err:
        break
    elif count == 20:
        break
    
    count+=1 # count only, when system is solved
    print(count)
    # update u
    u += u_d 
    
    #----------------------------------------------------#
    ud_array.append(u_d)
    normed_du = np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
    stepsize_du = normed_du # times alpha
    stepsizes.append(stepsize_du)
    #---------------------------------------------------#
    
ud_array.append(u_d) 
print('u= ' , u)
solutions.append(np.array(u))
iter_counts.append(count)

# print deformation plot (after first iteration)
get_defplot(u, patches_u, kvs_u, MP_u, n_el)

 
 
 1. loading: 0.0008886666666666666 

1
Residual = 0.001517200563820265
Energy   = 0.0
Error= 0.05238987770370996
 
 
 2. loading: 0.0017773333333333335 

1
Residual = 222.29491626237825
Energy   = 0.13987888731289125
Error= 0.4535755902229109
 
 
 3. loading: 0.002666 

1
Residual = 16.56352595543931
Energy   = 0.0174236142278912
Error= 0.17436209534307412
residual = 1.141198190305165
energy   = -0.0004435796962555051
Error= 0.07676071029398193
1
residual = 0.15000443633562816
energy   = -0.004079985637069748
Error= 0.04548924189040286
2
residual = 0.06123568087395462
energy   = -0.0052723447105086805
Error= 0.023358733831919866
3
residual = 0.003468292008589352
energy   = -0.0055901200651183695
Error= 0.007880229723980874
4
residual = 0.0001424904518132774
energy   = -0.005623526447551184
Error= 0.0012199020525153973
5
residual = 9.985939131558219e-08
energy   = -0.005624272011899843
Error= 3.653227998101389e-05
6
residual = 2.0208018430948424e-13
energy   = -0.005624272274469051


In [55]:
get_defplot_scalar(cauchystress(u, MP_block), patches_u, kvs_u, MP_u, n_el, geos)

In [56]:
get_defplot_scalar(local_vol(u, MP_block), patches_u, kvs_u, MP_u, n_el, geos)

In [57]:
get_defplot_evalp(cauchystress(u, MP_block), local_vol(u, MP_block), patches_u, kvs_u, MP_u, n_el)


 patch: 0
cauchystress_inner= 5.885988884586033e-05
cauchystress_outer= 0.00040673689628666695
volume_inner= 1.0066106126181402
volume_outer= 1.0586623702628999

 patch: 2
cauchystress_inner= 1.983996377828686e-05
cauchystress_outer= 1.1880133208920663e-05
volume_inner= 0.9968576402291517
volume_outer= 1.0020320339113924


In [58]:
from IPython.display import HTML

figsize(14, 4)
s_sol= shape(solutions)[0]
fields = [solutions[tt] for tt in range(s_sol)]
HTML(animate_field(fields, patches_u, kvs_u, MP_u, res=(50,70), interval=635, progress=True).to_html5_video())

5it [00:01,  4.94it/s]                                                                                                                                      


In [59]:
# solve linearized variational problem - iterative, without line-search

#initial value
u= np.zeros(dd)

epsilon= 1e-11# 1e-5

solutions = []
stepsizes = [] 
ud_array = []
iter_counts = []
vol= []


### Linear elasticity for largest loading
l_val = loading[-1] # take last loading value
neu_data = [(0,'left',gN, l_val), (1,'left',gN, l_val), (2,'left',gN, l_val), (3,'left',gN, l_val)] # set neumann

A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
#M = ass_mass(MP_block)
#Minv = make_solver(M)
r0 = np.transpose(b).dot(Minv.dot(b)) #L2-norm
#r0 = np.linalg.norm(b)


print("Norm of rhs for max loading: {}".format(r0))
print("Tolerance:                   {}".format(r0*epsilon))
###

# incremental loading # ----------------------------------
for t in range(len(loading)): # time steps
    print(" \n \n {}. loading: {} \n".format(t+1,loading[t]))
    
    # set Neumann data via incremental loading
    l_val = loading[t]
    neu_data = [(0,'left',gN, l_val), (1,'left',gN, l_val), (2,'left',gN, l_val), (3,'left',gN, l_val)] # set neumann

    count = 0
    while True:
        count+=1

        # Assemble matrices and rhs in every iteration step
        if count == 1:
            A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
            r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
            #r = np.linalg.norm(b)
            print('Residual =',r)
            print('Energy   =',J(u))
            
            w= np.inner(np.transpose(u_d),b)
            err = np.sqrt(np.abs(w))
            print('Error=', err)
        #else: # Use from last iteration; see below (*)
        #    A, b = ass_nonlinsystem_RN(u)
            
        print(count)

        # # solve system # #
        #u_d = make_solver(A).dot(b) 
        u_d = make_solver(A).dot(b)
        u += u_d            

        # Compute new non-linear residual, already to be used for next iteration (*)
        A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
        r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm 
        #r= np.linalg.norm(b)
        print('residual =', r)
        print('energy   =', J(u))
        w= np.inner(np.transpose(u_d),b)
        err = np.sqrt(np.abs(w))
        print('Error=', err)
        
        #----------------------------------------------------#
        ud_array.append(u_d)
        normed_du = np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
        #normed_du = np.linalg.norm(u_d)/np.sqrt(len(u_d))
        stepsize_du = normed_du # times alpha
        stepsizes.append(stepsize_du)
        #---------------------------------------------------#

        #if r < epsilon * r0: # break condition
        if err < epsilon:
            break
        elif count == 25:
            break
    #
    ud_array.append(u_d) 
    print('u= ' , u)
    solutions.append(np.array(u))
    iter_counts.append(count)
    
    loc_vol= local_vol (u, MP_block)
    vol.append(loc_vol)
    
        
# print deformation plot (after first iteration)
get_defplot(u, patches_u, kvs_u, MP_u, n_el)

Norm of rhs for max loading: 0.013654805074382386
Tolerance:                   1.3654805074382386e-13
 
 
 1. loading: 0.0008886666666666666 

Residual = 0.001517200563820265
Energy   = 0.0
Error= 9.870243063428806e-09
1
residual = 222.09214162296024
energy   = 0.14262280027051127
Error= 0.754414784925144
2
residual = 16.384850514009415
energy   = 0.019730969824055037
Error= 0.2320347046990165
3
residual = 0.9670325433676099
energy   = 0.002731475544463512
Error= 0.0835418226441754
4
residual = 0.06411634253852087
energy   = 0.00039253105692744876
Error= 0.028773818483201406
5
residual = 0.05935481226035497
energy   = -0.00030207292383950665
Error= 0.010255622314814715
6
residual = 0.0063724109779771925
energy   = -0.0006248161609234625
Error= 0.012945495003735796
7
residual = 0.004386009359192906
energy   = -0.0007174459858863151
Error= 0.004162387783556162
8
residual = 6.387045452509933e-05
energy   = -0.0007357757142970182
Error= 0.0022934572097053135
9
residual = 1.1357650160725646

In [60]:
iter_counts

[12, 5, 4]

In [61]:
sum(iter_counts)

21

In [62]:
get_defplotPP(u, patches_u, kvs_u, MP_u, n_el, r_in, r_out)


 patch: 0
inner radius_0= 2.800773923415906
outer radius_0= 3.089214588951762
displacement_inner= [ 0.85584153 -0.28882784]
displacement_outer= [ 0.82495362 -0.2964912 ]
dis_inner_x/dis_outer_x:  1.0374420043339927
dis_inner_y/dis_outer_y:  0.9741531537825165
 ratio_inner: 1.4511781986610912
 ratio_outer: 1.3729842617563388

 patch: 1
inner radius_0= 2.119797285776608
outer radius_0= 2.3937721110662653
displacement_inner= [ 0.18945613 -0.18131402]
displacement_outer= [ 0.13896933 -0.13973481]
dis_inner_x/dis_outer_x:  1.3632945888111963
dis_inner_y/dis_outer_y:  1.2975580544344736
 ratio_inner: 1.0983405625785534
 ratio_outer: 1.0638987160294513

 patch: 2
inner radius_0= 2.0165069671844638
outer radius_0= 2.3215597462232394
displacement_inner= [-0.08650682 -0.00076347]
displacement_outer= [-0.07155953 -0.00099374]
dis_inner_x/dis_outer_x:  1.2088790742084023
dis_inner_y/dis_outer_y:  0.7682814574593821
 ratio_inner: 1.0448222627898776
 ratio_outer: 1.0318043316547731

 patch: 3
inner

In [63]:
figsize(6,3)
fig, ax= plt.subplots()

s_cts= shape(iter_counts)[0]
plot(range(s_cts), iter_counts)

ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.title(' Number of iterations over loading process');
xlabel('Loading'); ylabel('Number of iterations');

In [64]:
s_sol= shape(solutions)[0]

In [65]:
s_sol


3

In [66]:
## animation

"""Visualization functions."""
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection


def animate_field(fields, patches_u, kvs_u, MP_u, vrange=None, res=(50,50),cmap=None, interval=50, progress=False, s_sol=3):
    """Animate a sequence of scalar fields over a geometry."""
    fields = list(fields)
 
    fig, ax = plt.subplots(figsize= (8,8))
    ax.set_xlim(left=-3.5, right=3.5)
    ax.set_ylim(bottom=-3.2, top=3.5)
    ax.set_aspect('equal')
    
    
    ar= np.linspace(1,3, s_sol)
    factor = ar[0]
    C = None
    vrange= None

    if np.isscalar(res):
        res = (res, res)

    # first solution
    #u= LS.complete(fields[0])
    u = fields[0]
    
    #Split solution vector into displacement components
    u1_funcs, u2_funcs = split_u(u, MP_u, kvs_u, patches_u)

    # evaluate displacement (and "pressure") over a grid in the parameter domain
    # visualization per patch
    for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches_u): #u_funcs 
        grd = tuple(np.linspace(s[0], s[1], r) for (s, r) in zip(geo.support, res))
        XY = geo.grid_eval(grd)
        dis1 = u1_func.grid_eval(grd) #x-value
        dis2 = u2_func.grid_eval(grd) #y-value
        dis = np.stack((dis1,dis2), axis=-1)
        if C is None:
            C = np.sqrt(np.power(dis[..., 0], 2) + np.power(dis[..., 1], 2))
        if vrange is None:
            #vrange = (0.0, 1.5e-3)
            vrange = (C.min(), C.max())
        #vis.plot_grid(XY[..., 0] + dis[..., 0], XY[..., 1] + dis[..., 1], ax=ax, color="black")
        quadmesh = plt.pcolormesh(XY[..., 0] + factor* dis[..., 0], XY[..., 1] + factor*dis[..., 1], C, shading='gouraud', cmap='viridis',
                                    vmin=vrange[0], vmax=vrange[1], axes=ax)
        
   
    fig.colorbar(quadmesh, ax=ax);
    tqdm = vis.utils.progress_bar(progress)
    pbar = tqdm(total=len(fields))
    
    def anim_func(i):
        plt.cla()
        ax.set_xlim(left=-3.5, right=3.5)
        ax.set_ylim(bottom=-3.2, top=3.5)
        ax.set_aspect('equal')
        #factor = ar[i] # choose factor for deformation plot
        #u = LS.complete(fields[i])
        vrange= None
        
        u = fields[i]
        u1_funcs, u2_funcs = split_u(u, MP_u, kvs_u, patches_u)

    # visualization per patch
        for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches_u): #u_funcs 
            grd = tuple(np.linspace(s[0], s[1], r) for (s, r) in zip(geo.support, res))
            XY = geo.grid_eval(grd)
            dis1 = u1_func.grid_eval(grd) #x-value
            dis2 = u2_func.grid_eval(grd) #y-value
            dis = np.stack((dis1,dis2), axis=-1)
            C = np.sqrt(np.power(dis[..., 0], 2) + np.power(dis[..., 1], 2))
            if vrange is None:
                #vrange = (0.0, 1.5e-3)
                vrange = (C.min(), C.max())
            quadmesh = plt.pcolormesh(XY[..., 0] + factor*dis[..., 0], XY[..., 1] + factor*dis[..., 1], C, shading='gouraud', cmap='viridis', 
                                      vmin=vrange[0], vmax=vrange[1], axes=ax)
        
            #quadmesh.set_array(C.ravel())
        pbar.update()
        if i == len(u) - 1:
            pbar.close()
  
    return animation.FuncAnimation(plt.gcf(), anim_func, frames=len(fields), interval=interval, repeat=False)


In [67]:
from IPython.display import HTML

figsize(14, 4)
s_sol= shape(solutions)[0]
fields = [solutions[tt] for tt in range(s_sol)]
HTML(animate_field(fields, patches_u, kvs_u, MP_u, res=(50,70), interval=635, progress=True).to_html5_video())

4it [00:00,  4.99it/s]                                                                                                                                      


In [33]:
plt.clf()

<Figure size 1400x400 with 0 Axes>

In [34]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection

plt.rcParams['animation.html'] = 'html5'
plt.rcParams['font.size'] = 18

fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect('equal')
res=(50,70)
#vrange=(0.0, 1.5e-3)

s_sol= shape(solutions)[0]
ims = []
ar= np.linspace(1,2, s_sol)
factor = 1


#fields = list(ud_array)
fields = list(solutions)

for tt in range(s_sol):
    #u = LS.complete(fields[tt])
    u = fields[tt]
    #factor = ar[tt] # choose factor for deformation plot
    vrange = None
    ims_q = []
    u1 = u[:MP_u.numdofs] 
    u2 = u[MP_u.numdofs:]

    # restrict solution to each individual patch - BSpline functions
    u1_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u1)
            for p in range(len(patches_u))]
    u2_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u2)
            for p in range(len(patches_u))]
    plt.ioff()
    
    # visualization per patch
    for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches_u): #u_funcs 
        im_q=[]
        grd = tuple(np.linspace(s[0], s[1], r) for (s, r) in zip(geo.support, res))
        XY = geo.grid_eval(grd)
        dis1 = u1_func.grid_eval(grd) #x-value
        dis2 = u2_func.grid_eval(grd) #y-value
        dis = np.stack((dis1,dis2), axis=-1)
        C = np.sqrt(np.power(dis[..., 0], 2) + np.power(dis[..., 1], 2))
        if vrange is None:
            #vrange = (0.0, 1.5e-3)
            vrange = (C.min(), C.max())
        im_q = plt.pcolormesh(XY[..., 0] + factor*dis[..., 0], XY[..., 1] + factor*dis[..., 1], C, shading='gouraud', cmap='viridis', 
                                    vmin=vrange[0], vmax=vrange[1], axes=ax) # shading for smoothing
        
        im_q.set_array(C.ravel()) 
        ims_q.append(im_q) 
    
    ims.append(ims_q)
    #print(shape(ims))
#print(ims[0])
    
#fig.colorbar(im_q, ax=ax);
colorbar();
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat= False) # repeat_delay=2000



#ani.to_html5_video()


Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'))
#ani.save("rtest.mp4", writer=writer )
vid = ani.to_html5_video()
# saving to m4 using ffmpeg writer
#writervideo = animation.FFMpegWriter(fps=500)
#ani.save('Robin_15.mp4', writer=writervideo)

plt.close()



In [35]:
ani

In [36]:
# solve linearized variational problem - iterative, without line-search

#initial value
u= np.zeros(dd)

epsilon= 1e-11# 1e-5
max_err=1e-11

solutions = []
stepsizes = [] 
ud_array = []
iter_counts = []


### Linear elasticity for largest loading
l_val = loading[-1] # take last loading value
neu_data = [(0,'left',gN, l_val), (1,'left',gN, l_val), (2,'left',gN, l_val), (3,'left',gN, l_val)] # set neumann

A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
#M = ass_mass(MP_block)
#Minv = make_solver(M)
r0 = np.transpose(b).dot(Minv.dot(b)) #L2-norm
#r0 = np.linalg.norm(b)


print("Norm of rhs for max loading: {}".format(r0))
print("Tolerance:                   {}".format(r0*epsilon))
###

# incremental loading # ----------------------------------
for t in range(len(loading)): # time steps
    print(" \n \n {}. loading: {} \n".format(t+1,loading[t]))
    
    # set Neumann data via incremental loading
    l_val = loading[t]
    neu_data = [(0,'left',gN, l_val), (1,'left',gN, l_val), (2,'left',gN, l_val), (3,'left',gN, l_val)] # set neumann

    count = 0
    while count <len(loading)-2:
        count+=1

        # Assemble matrices and rhs in every iteration step
        if count == 1:
            A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
            r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
            print('Residual =',r)
            print('Energy   =',J(u))
            
        #else: # Use from last iteration; see below (*)
            
        print(count)

        # # solve system # #
        #u_d = make_solver(A).dot(b) 
        u_d = make_solver(A).dot(b)
        u += u_d          

        # Compute new non-linear residual, already to be used for next iteration (*)
        A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
        r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
            
        print('residual =', r)
        print('energy   =', J(u))
        
        w= np.inner(np.transpose(u_d),b)
        err = np.sqrt(np.abs(w))
        print('Error=', err)
        
        ud_array.append(u_d)
        normed_du = np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
        #normed_du = np.linalg.norm(u_d)/np.sqrt(len(u_d))
        stepsize_du = normed_du # times alpha
        stepsizes.append(stepsize_du)
    solutions.append(np.array(u))
    iter_counts.append(count)
    count=0

while True:
    count+=1

    # Assemble matrices and rhs in every iteration step
    if count == 1:
        A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
        r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
        print('Residual =',r)
        print('Energy   =',J(u))
    #else: # Use from last iteration; see below (*)

    print(count)

    # # solve system # #
    #u_d = make_solver(A).dot(b) 
    u_d = make_solver(A).dot(b)
    u += u_d          

    # Compute new non-linear residual, already to be used for next iteration (*)
    A, b = ass_nonlinsystem_RN(u, MP_block, neu_data, AR)
    r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm

    print('residual =', r)
    print('energy   =', J(u))

    w= np.inner(np.transpose(u_d),b)
    err = np.sqrt(np.abs(w))
    print('Error=', err)

    #----------------------------------------------------#
    ud_array.append(u_d)
    normed_du = np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
    #normed_du = np.linalg.norm(u_d)/np.sqrt(len(u_d))
    stepsize_du = normed_du # times alpha
    stepsizes.append(stepsize_du)
    #iter_counts.append(count)
    #---------------------------------------------------#

    #if r < epsilon * r0: # break condition
    if err < max_err:
        break
    elif count == 20:
        break
    #
ud_array.append(u_d) 
print('u= ' , u)
solutions.append(np.array(u))
iter_counts.append(count)
        
# print deformation plot (after first iteration)
get_defplot(u, patches_u, kvs_u, MP_u, n_el)

Norm of rhs for max loading: 0.013654805074382386
Tolerance:                   1.3654805074382386e-13
 
 
 1. loading: 0.0008886666666666666 

Residual = 0.001517200563820265
Energy   = 0.0
1
residual = 222.09214162296024
energy   = 0.14262280027051127
Error= 0.754414784925144
 
 
 2. loading: 0.0017773333333333335 

Residual = 222.29491626237825
Energy   = 0.13987888731289125
1
residual = 16.509576422894483
energy   = 0.018903157126217988
Error= 0.23031716554561024
 
 
 3. loading: 0.002666 

Residual = 16.563525955439395
Energy   = 0.017423614227891192
1
residual = 1.1411981903051573
energy   = -0.00044357969625544896
Error= 0.08742149043601424
Residual = 1.1411981903051573
Energy   = -0.00044357969625544896
1
residual = 0.15000443633561764
energy   = -0.004079985637069675
Error= 0.04308912328915125
2
residual = 0.06123568087396533
energy   = -0.005272344710508658
Error= 0.017092857185825223
3
residual = 0.0034682920085889867
energy   = -0.005590120065118364
Error= 0.0113905629847430